**CS284_PA2: Conditional Random Fields for Hyphenated Words**

*Performed by: Marwin B. Alejo*
*October 18, 2021*

This Jupyter Notebook contain the Python scripts used to preprocess the Hyphenated English Dataset and to evaluate 



***Dataset Preprocessing***

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import random

def datasetParser(directory):
	dataset= []
	inputfile = open(directory)
	line = inputfile.readline().strip()

	while line:
		line = inputfile.readline().strip()
		dataset.append(line)

	inputfile.close()
	return dataset

def datasetSplit(dataset):
	test_dataset_raw = []
	train_dataset_raw = []

	random.shuffle(dataset)
	length = len(dataset)//2
	
	for i in range(length):
		line = random.choice(dataset)
		train_dataset_raw.append(line)
		dataset.remove(line)

	test_dataset_raw = dataset
	return train_dataset_raw, test_dataset_raw

def crfppFormatConv(dataset, directory):
	outputfile = open(directory,'w')

	for line in dataset:
		for i in range(len(line)):			
			if i == len(line) - 1: 
				outputfile.write(line[i] + '\t' + str(0) + '\n')
			
			elif line[i + 1] == '-':
				outputfile.write(line[i] + '\t' + str(1) + '\n')
			
			elif line[i] != '-' :
				outputfile.write(line[i] + '\t' + str(0) + '\n')

		outputfile.write('\n')
	outputfile.close()

def main(): 
	print('Begin processing...')
	directory = '/content/drive/MyDrive/Colab_Notebooks/English-Hyphenation-dataset.txt'
	
	print("Reading the dataset...")
	dataset = datasetParser(directory)
	
	print("Randomly splitting the dataset...")
	train_dataset, test_dataset = datasetSplit(dataset)

	print("Converting splitted raw dataset to CRF++ readable format...")
	crfppFormatConv(train_dataset, '/content/drive/MyDrive/Colab_Notebooks/EngHyphen_train.txt')
	crfppFormatConv(test_dataset, '/content/drive/MyDrive/Colab_Notebooks/EngHyphen_test.txt')

if __name__ == '__main__':
    main()
    print('Finish processing.')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Begin processing...
Reading the dataset...
Randomly splitting the dataset...
Converting splitted raw dataset to CRF++ readable format...
Finish processing.


***Evaluation***

In [ ]:
from __future__ import division

def readResults(filename):
  file = open(filename)
  line = file.readline()
  true_output = []
  predicted_output = []
  true_output_set = []
  predicted_output_set = []
  while line:
    if not line.strip():
      true_output = []
      predicted_output = []
      true_output_set.append(true_output)
      predicted_output_set.append(predicted_output)
    else:
      line = line.split()
      true_output.append(line[-2])
      predicted_output.append(line[-1])
    line = file.readline()	
  file.close()
  return true_output_set, predicted_output_set

def letterScore(true_output, predicted_output):
  TN, TP, FN, FP, total = 0, 0, 0, 0, 0
  size = len(true_output)
  for i in range(size):
    for j in range(len(true_output[i])):
      true = int(true_output[i][j])
      predicted = int(predicted_output[i][j])
      if true == predicted:
        TP = TP + 1
      elif true == 1 and predicted == 0:
        FN = FN + 1
      elif true == 0 and predicted == 1:
        FP = FP + 1
      elif true == 0 and predicted == 0:
        TN = TN + 1
      total = total + 1
  return TP, TN, FP, FN, total

def OWScore(true_output, predicted_output):
  owe, total = 0, 0
  size = len(true_output)
  
  for i in range(size):
    if true_output[i] != predicted_output[i]:
      owe = owe + 1
    total = total + 1
  return owe, total

def SWScore(true_output, predicted_output):
  swe, total = 0, 0
  size = len(true_output)
  for i in range(size):
    for j in range(len(true_output[i])):
      true = int(true_output[i][j])
      predicted = int(predicted_output[i][j])
      if true == 0 and predicted == 1:
        swe = swe + 1
        break
    total = total + 1
  return swe, total

def main():
  print('Begin processing...')
  filename = '/content/drive/MyDrive/Colab_Notebooks/EngHyphen_results.txt'
  true_output, predicted_output = readResults(filename)
  TP, TN, FP, FN, total = letterScore(true_output, predicted_output)
  precision = (TP + TN)
  print('n (total number of tested letters) = ', total, "letters")
  print("TP (True Positive) = ", TP,"letters", " (", round((TP/total)*100, 4),"%", ")")
  print("TN (True Negative) = ", TN,"letters" " (", round((TN/total)*100, 4),"%", ")")
  print("FP (False Positive) = ", FP,"letters" " (", round((FP/total)*100, 4),"%", ")")
  print("FN (False Negative) = ", FN,"letters" " (", round((FN/total)*100, 4),"%", ")")
  print("Overall Letter-level Error Rate: (FP + FN)/ (TP + TN + FP +FN) = ", round(((FP + FN)/ (TP + TN + FP + FN))*100,4),"%")
  print("Serious Letter-level Error Rate: (FP)/ (TP + TN + FP +FN) = ", round(((FP)/ (TP + TN + FP + FN))*100, 4),"%")
  print("Accuracy: (TP + TN)/ (TP + TN + FP +FN) = ", round(((TP + TN)/(TP + TN + FP + FN))*100,4),"%")
  owe, total = OWScore(true_output, predicted_output)
  print("Total no. of words = ", total)
  print("Overall Word-level Error = ", owe, "words")
  print("Overall Word-level Error Rate =", round((owe/total)*100,4),"%")
  swe, total = SWScore(true_output, predicted_output)
  print("Serious Word-level Error = ", swe, "words")
  print("Serious Word-level Error Rate =", round((swe/total)*100,4),"%")
  
if __name__ == '__main__':
  main()
  print('Finish processing.')

Begin processing...
n (total number of tested letters) =  275782 letters
TP (True Positive) =  270018 letters  ( 97.9099 % )
TN (True Negative) =  0 letters ( 0.0 % )
FP (False Positive) =  2764 letters ( 1.0022 % )
FN (False Negative) =  3000 letters ( 1.0878 % )
Overall Letter-level Error Rate: (FP + FN)/ (TP + TN + FP +FN) =  2.0901 %
Serious Letter-level Error Rate: (FP)/ (TP + TN + FP +FN) =  1.0022 %
Accuracy: (TP + TN)/ (TP + TN + FP +FN) =  97.9099 %
Total no. of words =  33001
Overall Word-level Error =  3158 words
Overall Word-level Error Rate = 9.5694 %
Serious Word-level Error =  2577 words
Serious Word-level Error Rate = 7.8089 %
Finish processing.
